In [1]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

/home/ubuntu/anaconda3/envs/celeba/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generating Empty Aggregator to be loaded 

setting = 'local'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar100"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/final/cifar100/fig3/local_defend/'
args_.validation = False
args_.tune_steps = None

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████| 100/100 [00:00<00:00, 401.01it/s]


===> Initializing clients..


100%|██████████| 100/100 [00:13<00:00,  7.17it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.613 | Train Acc: 0.958% |Test Loss: 4.612 | Test Acc: 1.167% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [3]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [4]:
# Import Model Weights
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/FedAvg_adv/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
#     args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy")
    weights = np.load(args_.save_path+"train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/figperturb/fedem_avg_p0_1/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
# Set trials 
num_trials = 50
batch_size = 64
adv_idx = 0
adv_idxs = [0,1]
# model_idx_max = 8
model_idx_max = 3

In [ ]:
dists_measure_legit = np.zeros([num_trials, model_idx_max])
dists_measure_adv = np.zeros([num_trials, model_idx_max])
dists_measure_adv_ensemble = np.zeros([num_trials, model_idx_max])


# attack parameters
eps = 4.5
iteration = 10
target = -1
eps_norm = 2
step_size = 0.01

for i in range(num_trials):
    print("num_trial:", i)
    t1 = Boundary_Transferer(models_list=models_test[:model_idx_max], dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = range(model_idx_max)

    t1.atk_params = IFSGM_Params()
    t1.atk_params.set_params(batch_size=1, eps=0.1, alpha=0.01, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    
    t1.set_adv_NN(t1.base_nn_idx)

    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = False,print_res = False)

    t1.set_ensemble_adv_NN(adv_idxs)
    base_ep_Eadv, victim_eps_Eadv = t1.ensemble_adversarial_direction(ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_Eadv.items():
        dists_measure_adv_ensemble[i,idx] = np.abs(base_ep_Eadv - value)
        idx+=1

In [ ]:
np.average(dists_measure_legit)

In [ ]:
np.average(dists_measure_adv)

In [ ]:
np.average(dists_measure_adv_ensemble)